In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [261]:
def softmax(t):
    res = np.exp(t)
    return res / np.sum(res)

def relu(t):
    return np.maximum(t, 0)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    return sigmoid(x) * (1-sigmoid(x))

def relu_deriv(t):
    return (t>=0).astype(float)

def to_one_hot_encoding(y, dim = 10):
    res = np.zeros([1, dim])
    res[0][y] = 1
    return res


class Perceptron:
    def __init__(self, input_dim, h1_dim, out_dim, learn_rate = 0.05):
        self.input_dim = input_dim
        self.h1_dim = h1_dim
        self.out_dim = out_dim
        self.learn_rate = learn_rate

        self.t1 = np.zeros([1, self.h1_dim])
        self.h1 = np.zeros([1, self.h1_dim])
        self.t2 = np.zeros([1, self.out_dim])
        self.out = np.zeros([1, self.out_dim])

        self.w1 = (np.random.rand(self.input_dim, self.h1_dim) - 0.5)*0.1
        self.b1 = (np.random.rand(1, self.h1_dim) - 0.5)*0.1
        self.w2 = (np.random.rand(self.h1_dim, self.out_dim) - 0.5)*0.1
        self.b2 = (np.random.rand(1, self.out_dim) - 0.5)*0.1
        self.drop_gradients()


    def drop_gradients(self):
        self.dE_dw1 = np.zeros((self.input_dim, self.h1_dim))
        self.dE_db1 = np.zeros((1, self.h1_dim))
        self.dE_dw2 = np.zeros((self.h1_dim, self.out_dim))
        self.dE_db2 = np.zeros((1, self.out_dim))


    def run(self, data, activation_function):
        self.t1 = data @ self.w1 + self.b1
        self.h1 = activation_function(self.t1)
        self.t2 = self.h1 @ self.w2 + self.b2
        self.out = self.t2
        return self.out


    def calculate_gradient(self, data, right_out, activation_deriv):
        dE_dt2 = self.out - right_out
        self.dE_dw2 += self.h1.T @ dE_dt2
        self.dE_db2 += dE_dt2

        dE_dh1 = dE_dt2 @ self.w2.T
        dE_dt1 = dE_dh1 * activation_deriv(self.t1)
        self.dE_dw1 += data * dE_dt1
        self.dE_db1 += dE_dt1


    def apply_gradient(self, batch_size=1):
        self.w1 -= self.learn_rate * self.dE_dw1 / batch_size
        self.w2 -= self.learn_rate * self.dE_dw2 / batch_size
        self.b1 -= self.learn_rate * self.dE_db1 / batch_size
        self.b2 -= self.learn_rate * self.dE_db2 / batch_size

        self.drop_gradients()



In [262]:
def train_model(model, X_train, y_train, X_test, y_test):
    for epoch in range (1, 2000):
        loss = 0
        inner_counter = 1
        for i in np.random.permutation(len(X_train)):
            prediction = model.run(data=X_train[i].flatten(), activation_function=relu)
            loss += np.square(y_train[i] - prediction[0][0])/2
            model.calculate_gradient(data=X_train[i], right_out=y_train[i].flatten(), activation_deriv=relu_deriv)
            #if inner_counter % 50 == 0:
            model.apply_gradient(batch_size=1)
            #inner_counter += 1

        test_loss = 0
        # for i in range(0, 10000):
        #     prediction = np.argmax(model.run(data=X_test[i], activation_function=relu))
        #     test_loss += (y_test[i] - prediction) ** 2
        print(f"Epoch №{epoch} finished with accuracy {round(loss/len(X_train), 6)} Test dataset accuracy {round(test_loss/len(X_train) * 100, 2)}")


In [263]:
import pandas as pd
P_START = 21
dataframe_sorb = pd.read_excel('Silica-loc-isoth1.xlsx', header=None, sheet_name="Adsorption")
AX_train = []
PX_train = []
pore_N = dataframe_sorb.shape[1]
AX_train = np.array(dataframe_sorb.iloc[0][1:])
PX_train = np.array(dataframe_sorb[0][P_START:])
ax_scale = np.max(AX_train)
px_scale = np.max(PX_train)
AX_train = AX_train / ax_scale
PX_train = PX_train / px_scale

X_train = np.empty(shape = (len(AX_train) * len(PX_train), 2))
Y_train = np.empty(shape =len(AX_train) * len(PX_train))
k = 0
for i in range(0, len(AX_train)):
    for j in range(0, len(PX_train)):
        X_train[k] = np.array([AX_train[i], PX_train[j]])
        Y_train[k] = dataframe_sorb[i+1][j+P_START]
        k+=1
y_scale = np.max(Y_train)
Y_train = Y_train / y_scale

In [269]:
X_train = np.linspace(-10, 10, 100)
Y_train = np.square(X_train)*2 + np.power(X_train, 3)*0.5 + np.sin(X_train)*0.2 + np.cos(X_train*1)*100
x_scale = np.max(X_train)
y_scale = np.max(Y_train)
X_train = X_train / x_scale
Y_train = Y_train / y_scale

In [270]:
np.random.seed(0)
model = Perceptron(input_dim=1, h1_dim=1000, out_dim=1)

In [271]:
model.learn_rate = 0.05
train_model(model, X_train, Y_train, X_train, Y_train)

Epoch №1 finished with accuracy 0.036182 Test dataset accuracy 0.0
Epoch №2 finished with accuracy 0.018782 Test dataset accuracy 0.0
Epoch №3 finished with accuracy 0.017045 Test dataset accuracy 0.0
Epoch №4 finished with accuracy 0.016161 Test dataset accuracy 0.0
Epoch №5 finished with accuracy 0.015673 Test dataset accuracy 0.0
Epoch №6 finished with accuracy 0.015418 Test dataset accuracy 0.0
Epoch №7 finished with accuracy 0.0146 Test dataset accuracy 0.0
Epoch №8 finished with accuracy 0.014358 Test dataset accuracy 0.0
Epoch №9 finished with accuracy 0.013989 Test dataset accuracy 0.0
Epoch №10 finished with accuracy 0.013353 Test dataset accuracy 0.0
Epoch №11 finished with accuracy 0.012769 Test dataset accuracy 0.0
Epoch №12 finished with accuracy 0.012524 Test dataset accuracy 0.0
Epoch №13 finished with accuracy 0.012548 Test dataset accuracy 0.0
Epoch №14 finished with accuracy 0.011602 Test dataset accuracy 0.0
Epoch №15 finished with accuracy 0.011696 Test dataset accu

In [272]:
i = -1
a = []
right_answer = model.run(data=X_train[i].flatten(), activation_function=relu)
#dataframe_sorb = pd.read_excel('Silica-loc-isoth1.xlsx', header=None, sheet_name="Adsorption")
# data_sorb = dataframe_sorb.to_numpy()
# pressures_d = data_sorb[:, 0][21:]
# plt.plot(pressures_d, y_train[i] * y_scale, marker=".")
# plt.plot(pressures_d, right_answer.reshape(458, 1) * y_scale , marker=".")
print(right_answer*y_scale, Y_train[i]*y_scale)
# plt.show()

[[546.30927192]] 615.9840428701768


In [273]:
right_answer = []
for i in X_train:
    right_answer.append(model.run(data=i.flatten(), activation_function=relu)[0][0]*y_scale)
plt.plot(X_train*10, Y_train*y_scale, marker=".")
plt.plot(X_train*10, right_answer, marker=".")

In [ ]:
import PyQt5
%matplotlib notebook

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(AX_train, PX_train)
Z = Y_train.reshape(X.shape)
surf = ax.plot_surface(X, Y, Z)
plt.show()